# Feature Engineering for Air Quality Prediction

This notebook demonstrates the feature engineering pipeline for creating advanced features from air quality data.

## Objectives
1. Create temporal features (year, month, day, season, cyclical encoding)
2. Generate lag features (previous day values)
3. Build rolling averages (3-day, 7-day, 14-day)
4. Create ratio features (PM2.5/PM10 ratio)
5. Develop interaction features
6. Select optimal features for modeling

## Feature Categories
- **Temporal Features**: Time-based features with cyclical encoding
- **Lag Features**: Previous day values for all pollutants
- **Rolling Features**: Moving averages and standard deviations
- **Ratio Features**: Relationships between pollutants
- **Interaction Features**: Combined effects of multiple variables


In [3]:
# Install required packages if not already installed
import subprocess
import sys

def install_package(package):
    """Install package using pip if not already installed."""
    try:
        __import__(package)
        print(f"{package} is already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install required packages
required_packages = [
    "pandas",
    "numpy", 
    "matplotlib",
    "seaborn",
    "scikit-learn",
    "scipy",
    "tqdm",
    "joblib"
]

for package in required_packages:
    install_package(package)

# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
import warnings
import os

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

print("Feature engineering libraries imported successfully!")


pandas is already installed
numpy is already installed
matplotlib is already installed
seaborn is already installed
Installing scikit-learn...
scipy is already installed
tqdm is already installed
joblib is already installed
Feature engineering libraries imported successfully!


## 1. Data Loading and Feature Engineering Setup


In [4]:
# Load processed data
data_path = "../data/processed/"

print("🔍 Loading processed data for feature engineering...")
print(f"Data path: {data_path}")

# Load the cleaned dataset from preprocessing
try:
    df_processed = pd.read_csv(data_path + "cleaned_data.csv")
    print(f"✅ Loaded processed data successfully!")
    print(f"📊 Dataset shape: {df_processed.shape}")
    print(f"📋 Columns: {list(df_processed.columns)}")
    
    # Display basic info
    print(f"\n📈 Basic Information:")
    print(f"  Memory usage: {df_processed.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"  Date range: {df_processed['Date'].min()} to {df_processed['Date'].max()}")
    
    # Check cities
    if 'City' in df_processed.columns:
        city_counts = df_processed['City'].value_counts()
        print(f"  Cities: {dict(city_counts)}")
    
    # Check AQI
    if 'AQI' in df_processed.columns:
        print(f"  AQI range: {df_processed['AQI'].min():.2f} to {df_processed['AQI'].max():.2f}")
        print(f"  AQI mean: {df_processed['AQI'].mean():.2f}")
    
except Exception as e:
    print(f"❌ Error loading processed data: {e}")
    df_processed = pd.DataFrame()

# Import feature engineering class
import sys
sys.path.append('../src')
from feature_engineering import AirQualityFeatureEngineer

# Initialize feature engineer
if not df_processed.empty:
    # Save a temporary copy for the feature engineer
    df_processed.to_csv(data_path + "temp_processed_data.csv", index=False)
    feature_engineer = AirQualityFeatureEngineer(data_path + "temp_processed_data.csv")
    print("✅ Feature engineer initialized with processed data!")
else:
    print("⚠️ Feature engineer initialized with placeholder path")
    feature_engineer = AirQualityFeatureEngineer(data_path + "cleaned_data.csv")


🔍 Loading processed data for feature engineering...
Data path: ../data/processed/
✅ Loaded processed data successfully!
📊 Dataset shape: (7688, 16)
📋 Columns: ['City', 'Date', 'PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3', 'Benzene', 'Toluene', 'Xylene', 'AQI', 'AQI_Bucket']

📈 Basic Information:
  Memory usage: 2.04 MB
  Date range: 2015-01-01 to 2020-07-01
  Cities: {'Delhi': np.int64(1999), 'Chennai': np.int64(1884), 'Hyderabad': np.int64(1880), 'Visakhapatnam': np.int64(1171), 'Kolkata': np.int64(754)}
  AQI range: 22.00 to 346.50
  AQI mean: 148.85


INFO:feature_engineering:Initialized AirQualityFeatureEngineer with data path: ../data/processed/temp_processed_data.csv


✅ Feature engineer initialized with processed data!


## 2. Create Temporal Features


In [5]:
# Create temporal features from Date column
print("📅 CREATING TEMPORAL FEATURES")
print("=" * 50)

if not df_processed.empty and 'Date' in df_processed.columns:
    # Make a copy for feature engineering
    df_features = df_processed.copy()
    
    # Convert Date to datetime if not already
    df_features['Date'] = pd.to_datetime(df_features['Date'])
    
    # Extract basic temporal features
    df_features['year'] = df_features['Date'].dt.year
    df_features['month'] = df_features['Date'].dt.month
    df_features['day'] = df_features['Date'].dt.day
    df_features['day_of_week'] = df_features['Date'].dt.dayofweek  # 0=Monday, 6=Sunday
    df_features['day_of_year'] = df_features['Date'].dt.dayofyear
    df_features['week_of_year'] = df_features['Date'].dt.isocalendar().week
    
    # Create season feature
    season_mapping = {
        12: 'Winter', 1: 'Winter', 2: 'Winter',
        3: 'Spring', 4: 'Spring', 5: 'Spring',
        6: 'Summer', 7: 'Summer', 8: 'Summer',
        9: 'Fall', 10: 'Fall', 11: 'Fall'
    }
    df_features['season'] = df_features['month'].map(season_mapping)
    
    # Create cyclical encoding for temporal features
    # This helps the model understand that December is close to January
    df_features['month_sin'] = np.sin(2 * np.pi * df_features['month'] / 12)
    df_features['month_cos'] = np.cos(2 * np.pi * df_features['month'] / 12)
    df_features['day_of_week_sin'] = np.sin(2 * np.pi * df_features['day_of_week'] / 7)
    df_features['day_of_week_cos'] = np.cos(2 * np.pi * df_features['day_of_week'] / 7)
    df_features['day_of_year_sin'] = np.sin(2 * np.pi * df_features['day_of_year'] / 365)
    df_features['day_of_year_cos'] = np.cos(2 * np.pi * df_features['day_of_year'] / 365)
    
    # Create weekend indicator
    df_features['is_weekend'] = (df_features['day_of_week'] >= 5).astype(int)
    
    # Create month-end indicator (last 3 days of month)
    df_features['is_month_end'] = (df_features['day'] >= 29).astype(int)
    
    temporal_features = [
        'year', 'month', 'day', 'day_of_week', 'day_of_year', 'week_of_year',
        'season', 'month_sin', 'month_cos', 'day_of_week_sin', 'day_of_week_cos',
        'day_of_year_sin', 'day_of_year_cos', 'is_weekend', 'is_month_end'
    ]
    
    print(f"✅ Created {len(temporal_features)} temporal features")
    print(f"Temporal features: {temporal_features}")
    
    # Display temporal feature summary
    print(f"\n📊 Temporal Feature Summary:")
    print(f"  Date range: {df_features['Date'].min()} to {df_features['Date'].max()}")
    print(f"  Years: {sorted(df_features['year'].unique())}")
    print(f"  Seasons: {df_features['season'].value_counts().to_dict()}")
    print(f"  Weekend ratio: {df_features['is_weekend'].mean():.2f}")
    
else:
    print("⚠️ Cannot create temporal features - Date column not found")
    df_features = df_processed.copy()
    temporal_features = []

print(f"\n📈 Dataset shape after temporal features: {df_features.shape}")


📅 CREATING TEMPORAL FEATURES
✅ Created 15 temporal features
Temporal features: ['year', 'month', 'day', 'day_of_week', 'day_of_year', 'week_of_year', 'season', 'month_sin', 'month_cos', 'day_of_week_sin', 'day_of_week_cos', 'day_of_year_sin', 'day_of_year_cos', 'is_weekend', 'is_month_end']

📊 Temporal Feature Summary:
  Date range: 2015-01-01 00:00:00 to 2020-07-01 00:00:00
  Years: [np.int32(2015), np.int32(2016), np.int32(2017), np.int32(2018), np.int32(2019), np.int32(2020)]
  Seasons: {'Spring': 2089, 'Winter': 1921, 'Summer': 1871, 'Fall': 1807}
  Weekend ratio: 0.29

📈 Dataset shape after temporal features: (7688, 31)


## 3. Create Lag Features (Previous Day Values)


In [6]:
# Create lag features for pollutants and AQI
print("⏰ CREATING LAG FEATURES")
print("=" * 50)

if not df_features.empty and 'City' in df_features.columns:
    # Sort data by city and date for proper lag calculation
    df_features = df_features.sort_values(['City', 'Date']).reset_index(drop=True)
    
    # Define pollutant columns (excluding target variable AQI for now)
    pollutant_columns = [col for col in df_features.columns 
                        if col in ['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3', 'Benzene', 'Toluene']]
    
    # Define lag days to create
    lag_days = [1, 2, 3, 7]  # Previous 1, 2, 3, and 7 days
    
    print(f"Creating lag features for {len(pollutant_columns)} pollutants:")
    print(f"Pollutants: {pollutant_columns}")
    print(f"Lag days: {lag_days}")
    
    lag_features = []
    
    # Create lag features for pollutants
    for pollutant in pollutant_columns:
        if pollutant in df_features.columns:
            for lag in lag_days:
                lag_col_name = f'prev_{lag}d_{pollutant}'
                df_features[lag_col_name] = df_features.groupby('City')[pollutant].shift(lag)
                lag_features.append(lag_col_name)
                print(f"  ✅ {lag_col_name}")
    
    # Create lag features for AQI (target variable)
    if 'AQI' in df_features.columns:
        for lag in lag_days:
            lag_col_name = f'prev_{lag}d_AQI'
            df_features[lag_col_name] = df_features.groupby('City')['AQI'].shift(lag)
            lag_features.append(lag_col_name)
            print(f"  ✅ {lag_col_name}")
    
    print(f"\n✅ Created {len(lag_features)} lag features")
    
    # Check for missing values in lag features
    lag_missing = df_features[lag_features].isnull().sum()
    lag_missing = lag_missing[lag_missing > 0]
    
    if len(lag_missing) > 0:
        print(f"\n⚠️ Missing values in lag features (first few days for each city):")
        for col, count in lag_missing.head(10).items():
            print(f"  {col}: {count} missing values")
    else:
        print(f"\n✅ No missing values in lag features")
    
    # Display lag feature summary
    print(f"\n📊 Lag Feature Summary:")
    print(f"  Total lag features: {len(lag_features)}")
    print(f"  Pollutants with lag features: {len(pollutant_columns)}")
    print(f"  Lag days: {lag_days}")
    
else:
    print("⚠️ Cannot create lag features - City column not found")
    lag_features = []

print(f"\n📈 Dataset shape after lag features: {df_features.shape}")


⏰ CREATING LAG FEATURES
Creating lag features for 11 pollutants:
Pollutants: ['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3', 'Benzene', 'Toluene']
Lag days: [1, 2, 3, 7]
  ✅ prev_1d_PM2.5
  ✅ prev_2d_PM2.5
  ✅ prev_3d_PM2.5
  ✅ prev_7d_PM2.5
  ✅ prev_1d_PM10
  ✅ prev_2d_PM10
  ✅ prev_3d_PM10
  ✅ prev_7d_PM10
  ✅ prev_1d_NO
  ✅ prev_2d_NO
  ✅ prev_3d_NO
  ✅ prev_7d_NO
  ✅ prev_1d_NO2
  ✅ prev_2d_NO2
  ✅ prev_3d_NO2
  ✅ prev_7d_NO2
  ✅ prev_1d_NOx
  ✅ prev_2d_NOx
  ✅ prev_3d_NOx
  ✅ prev_7d_NOx
  ✅ prev_1d_NH3
  ✅ prev_2d_NH3
  ✅ prev_3d_NH3
  ✅ prev_7d_NH3
  ✅ prev_1d_CO
  ✅ prev_2d_CO
  ✅ prev_3d_CO
  ✅ prev_7d_CO
  ✅ prev_1d_SO2
  ✅ prev_2d_SO2
  ✅ prev_3d_SO2
  ✅ prev_7d_SO2
  ✅ prev_1d_O3
  ✅ prev_2d_O3
  ✅ prev_3d_O3
  ✅ prev_7d_O3
  ✅ prev_1d_Benzene
  ✅ prev_2d_Benzene
  ✅ prev_3d_Benzene
  ✅ prev_7d_Benzene
  ✅ prev_1d_Toluene
  ✅ prev_2d_Toluene
  ✅ prev_3d_Toluene
  ✅ prev_7d_Toluene
  ✅ prev_1d_AQI
  ✅ prev_2d_AQI
  ✅ prev_3d_AQI
  ✅ prev_7d_AQI

✅ Created 48 

## 4. Create Rolling Features (Moving Averages)


In [7]:
# Create rolling average and standard deviation features
print("📈 CREATING ROLLING FEATURES")
print("=" * 50)

if not df_features.empty and 'City' in df_features.columns:
    # Define rolling windows
    rolling_windows = [3, 7, 14]  # 3-day, 7-day, and 14-day rolling windows
    
    print(f"Creating rolling features with windows: {rolling_windows}")
    
    rolling_features = []
    
    # Create rolling features for pollutants
    for pollutant in pollutant_columns:
        if pollutant in df_features.columns:
            for window in rolling_windows:
                # Rolling mean
                mean_col = f'{window}d_avg_{pollutant}'
                df_features[mean_col] = df_features.groupby('City')[pollutant].rolling(
                    window=window, min_periods=1
                ).mean().reset_index(0, drop=True)
                rolling_features.append(mean_col)
                
                # Rolling standard deviation
                std_col = f'{window}d_std_{pollutant}'
                df_features[std_col] = df_features.groupby('City')[pollutant].rolling(
                    window=window, min_periods=1
                ).std().reset_index(0, drop=True)
                rolling_features.append(std_col)
                
                # Rolling maximum
                max_col = f'{window}d_max_{pollutant}'
                df_features[max_col] = df_features.groupby('City')[pollutant].rolling(
                    window=window, min_periods=1
                ).max().reset_index(0, drop=True)
                rolling_features.append(max_col)
                
                print(f"  ✅ {pollutant}: {window}d avg, std, max")
    
    # Create rolling features for AQI
    if 'AQI' in df_features.columns:
        for window in rolling_windows:
            # Rolling mean for AQI
            aqi_mean_col = f'{window}d_avg_AQI'
            df_features[aqi_mean_col] = df_features.groupby('City')['AQI'].rolling(
                window=window, min_periods=1
            ).mean().reset_index(0, drop=True)
            rolling_features.append(aqi_mean_col)
            
            print(f"  ✅ AQI: {window}d average")
    
    print(f"\n✅ Created {len(rolling_features)} rolling features")
    
    # Display rolling feature summary
    print(f"\n📊 Rolling Feature Summary:")
    print(f"  Total rolling features: {len(rolling_features)}")
    print(f"  Rolling windows: {rolling_windows}")
    print(f"  Statistics per window: mean, std, max")
    
    # Check for missing values in rolling features
    rolling_missing = df_features[rolling_features].isnull().sum()
    rolling_missing = rolling_missing[rolling_missing > 0]
    
    if len(rolling_missing) > 0:
        print(f"\n⚠️ Missing values in rolling features:")
        for col, count in rolling_missing.head(10).items():
            print(f"  {col}: {count} missing values")
    else:
        print(f"\n✅ No missing values in rolling features")
    
else:
    print("⚠️ Cannot create rolling features - City column not found")
    rolling_features = []

print(f"\n📈 Dataset shape after rolling features: {df_features.shape}")


📈 CREATING ROLLING FEATURES
Creating rolling features with windows: [3, 7, 14]
  ✅ PM2.5: 3d avg, std, max
  ✅ PM2.5: 7d avg, std, max
  ✅ PM2.5: 14d avg, std, max
  ✅ PM10: 3d avg, std, max
  ✅ PM10: 7d avg, std, max
  ✅ PM10: 14d avg, std, max
  ✅ NO: 3d avg, std, max
  ✅ NO: 7d avg, std, max
  ✅ NO: 14d avg, std, max
  ✅ NO2: 3d avg, std, max
  ✅ NO2: 7d avg, std, max
  ✅ NO2: 14d avg, std, max
  ✅ NOx: 3d avg, std, max
  ✅ NOx: 7d avg, std, max
  ✅ NOx: 14d avg, std, max
  ✅ NH3: 3d avg, std, max
  ✅ NH3: 7d avg, std, max
  ✅ NH3: 14d avg, std, max
  ✅ CO: 3d avg, std, max
  ✅ CO: 7d avg, std, max
  ✅ CO: 14d avg, std, max
  ✅ SO2: 3d avg, std, max
  ✅ SO2: 7d avg, std, max
  ✅ SO2: 14d avg, std, max
  ✅ O3: 3d avg, std, max
  ✅ O3: 7d avg, std, max
  ✅ O3: 14d avg, std, max
  ✅ Benzene: 3d avg, std, max
  ✅ Benzene: 7d avg, std, max
  ✅ Benzene: 14d avg, std, max
  ✅ Toluene: 3d avg, std, max
  ✅ Toluene: 7d avg, std, max
  ✅ Toluene: 14d avg, std, max
  ✅ AQI: 3d average
  ✅ AQI:

## 5. Create Ratio Features


In [8]:
# Create ratio features between different pollutants
print("🔗 CREATING RATIO FEATURES")
print("=" * 50)

ratio_features = []

# PM2.5/PM10 ratio - indicates fine vs coarse particles
if 'PM2.5' in df_features.columns and 'PM10' in df_features.columns:
    df_features['PM25_PM10_ratio'] = np.where(
        df_features['PM10'] != 0, 
        df_features['PM2.5'] / df_features['PM10'], 
        0
    )
    ratio_features.append('PM25_PM10_ratio')
    print("✅ PM25_PM10_ratio - Fine vs coarse particle ratio")

# NO2/NO ratio - indicates atmospheric oxidation
if 'NO2' in df_features.columns and 'NO' in df_features.columns:
    df_features['NO2_NO_ratio'] = np.where(
        df_features['NO'] != 0, 
        df_features['NO2'] / df_features['NO'], 
        0
    )
    ratio_features.append('NO2_NO_ratio')
    print("✅ NO2_NO_ratio - Atmospheric oxidation indicator")

# SO2/NO2 ratio - indicates pollution source type
if 'SO2' in df_features.columns and 'NO2' in df_features.columns:
    df_features['SO2_NO2_ratio'] = np.where(
        df_features['NO2'] != 0, 
        df_features['SO2'] / df_features['NO2'], 
        0
    )
    ratio_features.append('SO2_NO2_ratio')
    print("✅ SO2_NO2_ratio - Pollution source type indicator")

# CO/PM2.5 ratio - indicates combustion vs particulate pollution
if 'CO' in df_features.columns and 'PM2.5' in df_features.columns:
    df_features['CO_PM25_ratio'] = np.where(
        df_features['PM2.5'] != 0, 
        df_features['CO'] / df_features['PM2.5'], 
        0
    )
    ratio_features.append('CO_PM25_ratio')
    print("✅ CO_PM25_ratio - Combustion vs particulate pollution")

# O3/NO2 ratio - indicates photochemical smog
if 'O3' in df_features.columns and 'NO2' in df_features.columns:
    df_features['O3_NO2_ratio'] = np.where(
        df_features['NO2'] != 0, 
        df_features['O3'] / df_features['NO2'], 
        0
    )
    ratio_features.append('O3_NO2_ratio')
    print("✅ O3_NO2_ratio - Photochemical smog indicator")

# Benzene/Toluene ratio - indicates aromatic compound source
if 'Benzene' in df_features.columns and 'Toluene' in df_features.columns:
    df_features['Benzene_Toluene_ratio'] = np.where(
        df_features['Toluene'] != 0, 
        df_features['Benzene'] / df_features['Toluene'], 
        0
    )
    ratio_features.append('Benzene_Toluene_ratio')
    print("✅ Benzene_Toluene_ratio - Aromatic compound source")

# NH3/NOx ratio - indicates agricultural vs traffic pollution
if 'NH3' in df_features.columns and 'NOx' in df_features.columns:
    df_features['NH3_NOx_ratio'] = np.where(
        df_features['NOx'] != 0, 
        df_features['NH3'] / df_features['NOx'], 
        0
    )
    ratio_features.append('NH3_NOx_ratio')
    print("✅ NH3_NOx_ratio - Agricultural vs traffic pollution")

print(f"\n✅ Created {len(ratio_features)} ratio features")
print(f"Ratio features: {ratio_features}")

# Display ratio feature statistics
print(f"\n📊 Ratio Feature Statistics:")
for ratio_feature in ratio_features:
    if ratio_feature in df_features.columns:
        stats = df_features[ratio_feature].describe()
        print(f"  {ratio_feature}:")
        print(f"    Range: {stats['min']:.3f} to {stats['max']:.3f}")
        print(f"    Mean: {stats['mean']:.3f}, Std: {stats['std']:.3f}")

print(f"\n📈 Dataset shape after ratio features: {df_features.shape}")


🔗 CREATING RATIO FEATURES
✅ PM25_PM10_ratio - Fine vs coarse particle ratio
✅ NO2_NO_ratio - Atmospheric oxidation indicator
✅ SO2_NO2_ratio - Pollution source type indicator
✅ CO_PM25_ratio - Combustion vs particulate pollution
✅ O3_NO2_ratio - Photochemical smog indicator
✅ Benzene_Toluene_ratio - Aromatic compound source
✅ NH3_NOx_ratio - Agricultural vs traffic pollution

✅ Created 7 ratio features
Ratio features: ['PM25_PM10_ratio', 'NO2_NO_ratio', 'SO2_NO2_ratio', 'CO_PM25_ratio', 'O3_NO2_ratio', 'Benzene_Toluene_ratio', 'NH3_NOx_ratio']

📊 Ratio Feature Statistics:
  PM25_PM10_ratio:
    Range: 0.051 to 151.952
    Mean: 0.534, Std: 2.134
  NO2_NO_ratio:
    Range: 0.061 to 67.125
    Mean: 3.270, Std: 2.942
  SO2_NO2_ratio:
    Range: 0.022 to 25.647
    Mean: 0.424, Std: 0.690
  CO_PM25_ratio:
    Range: 0.000 to 0.451
    Mean: 0.019, Std: 0.016
  O3_NO2_ratio:
    Range: 0.033 to 473.118
    Mean: 1.755, Std: 6.630
  Benzene_Toluene_ratio:
    Range: 0.000 to 166.333
    Mea

## 6. Feature Summary and Save Engineered Dataset


In [9]:
# Feature engineering summary and save dataset
print("📊 FEATURE ENGINEERING SUMMARY")
print("=" * 60)

# Calculate total engineered features
all_engineered_features = temporal_features + lag_features + rolling_features + ratio_features
total_engineered = len(all_engineered_features)

print(f"📈 Feature Engineering Results:")
print(f"  Original features: {len(df_processed.columns)}")
print(f"  Engineered features: {total_engineered}")
print(f"  Total features: {len(df_features.columns)}")
print(f"  Feature increase: {len(df_features.columns) - len(df_processed.columns)}")

print(f"\n🔧 Feature Categories:")
print(f"  📅 Temporal features: {len(temporal_features)}")
print(f"  ⏰ Lag features: {len(lag_features)}")
print(f"  📈 Rolling features: {len(rolling_features)}")
print(f"  🔗 Ratio features: {len(ratio_features)}")

print(f"\n📊 Dataset Information:")
print(f"  Shape: {df_features.shape}")
print(f"  Memory usage: {df_features.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"  Date range: {df_features['Date'].min()} to {df_features['Date'].max()}")

# Check for missing values in engineered features
engineered_missing = df_features[all_engineered_features].isnull().sum()
engineered_missing = engineered_missing[engineered_missing > 0]

if len(engineered_missing) > 0:
    print(f"\n⚠️ Missing values in engineered features:")
    for col, count in engineered_missing.head(10).items():
        percentage = (count / len(df_features)) * 100
        print(f"  {col}: {count:,} ({percentage:.2f}%)")
else:
    print(f"\n✅ No missing values in engineered features")

# Save engineered dataset
print(f"\n💾 SAVING ENGINEERED DATASET")
print("=" * 50)

# Create output directory if it doesn't exist
output_dir = "../data/features/"
os.makedirs(output_dir, exist_ok=True)

# Save the engineered dataset
output_file = output_dir + "engineered_features.csv"
df_features.to_csv(output_file, index=False)

print(f"✅ Engineered dataset saved to: {output_file}")
print(f"   Shape: {df_features.shape}")
print(f"   File size: {os.path.getsize(output_file) / 1024**2:.2f} MB")

# Save feature metadata
metadata = {
    'temporal_features': temporal_features,
    'lag_features': lag_features,
    'rolling_features': rolling_features,
    'ratio_features': ratio_features,
    'total_engineered_features': total_engineered,
    'dataset_shape': df_features.shape,
    'date_range': f"{df_features['Date'].min()} to {df_features['Date'].max()}"
}

# Save metadata as a summary
metadata_file = output_dir + "feature_metadata.txt"
with open(metadata_file, 'w') as f:
    f.write("FEATURE ENGINEERING METADATA\n")
    f.write("=" * 50 + "\n\n")
    f.write(f"Total Engineered Features: {total_engineered}\n")
    f.write(f"Dataset Shape: {df_features.shape}\n")
    f.write(f"Date Range: {metadata['date_range']}\n\n")
    
    f.write("TEMPORAL FEATURES:\n")
    for feature in temporal_features:
        f.write(f"  - {feature}\n")
    
    f.write(f"\nLAG FEATURES ({len(lag_features)}):\n")
    for feature in lag_features:
        f.write(f"  - {feature}\n")
    
    f.write(f"\nROLLING FEATURES ({len(rolling_features)}):\n")
    for feature in rolling_features:
        f.write(f"  - {feature}\n")
    
    f.write(f"\nRATIO FEATURES ({len(ratio_features)}):\n")
    for feature in ratio_features:
        f.write(f"  - {feature}\n")

print(f"✅ Feature metadata saved to: {metadata_file}")

print(f"\n🎉 FEATURE ENGINEERING COMPLETED SUCCESSFULLY!")
print(f"📈 Dataset transformed from {len(df_processed.columns)} to {len(df_features.columns)} features")
print(f"🚀 Ready for model training! Next step: 04_model_training.ipynb")


📊 FEATURE ENGINEERING SUMMARY
📈 Feature Engineering Results:
  Original features: 16
  Engineered features: 172
  Total features: 188
  Feature increase: 172

🔧 Feature Categories:
  📅 Temporal features: 15
  ⏰ Lag features: 48
  📈 Rolling features: 102
  🔗 Ratio features: 7

📊 Dataset Information:
  Shape: (7688, 188)
  Memory usage: 11.92 MB
  Date range: 2015-01-01 00:00:00 to 2020-07-01 00:00:00

⚠️ Missing values in engineered features:
  prev_1d_PM2.5: 25 (0.33%)
  prev_2d_PM2.5: 30 (0.39%)
  prev_3d_PM2.5: 35 (0.46%)
  prev_7d_PM2.5: 55 (0.72%)
  prev_1d_PM10: 1,914 (24.90%)
  prev_2d_PM10: 1,919 (24.96%)
  prev_3d_PM10: 1,924 (25.03%)
  prev_7d_PM10: 1,944 (25.29%)
  prev_1d_NO: 23 (0.30%)
  prev_2d_NO: 28 (0.36%)

💾 SAVING ENGINEERED DATASET
✅ Engineered dataset saved to: ../data/features/engineered_features.csv
   Shape: (7688, 188)
   File size: 14.75 MB
✅ Feature metadata saved to: ../data/features/feature_metadata.txt

🎉 FEATURE ENGINEERING COMPLETED SUCCESSFULLY!
📈 Datase